# VaultX — Core Cryptographic Engine

This notebook explains the internal cryptographic design of **VaultX**.

VaultX is a user-space file encryption utility built on modern cryptographic primitives:
- **Argon2** for password-based key derivation
- **AES-256-GCM** for authenticated encryption

The goal is secure, auditable, and deterministic file protection.

In [ ]:
import os
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from argon2.low_level import hash_secret_raw, Type

KEY_LEN = 32
SALT_LEN = 16
NONCE_LEN = 12

## Key Derivation (Argon2)

VaultX does not store encryption keys.
Instead, it derives a cryptographic key from a user password using **Argon2**.

**Why Argon2?**
- Memory-hard to resist GPU/ASIC attacks
- Winner of the Password Hashing Competition

**Why Salt?**
- Prevents rainbow-table attacks
- Ensures unique keys for identical passwords

**Why memory-hard KDFs matter?**
- Increases attacker cost dramatically


In [ ]:
def derive_key(password: str, salt: bytes) -> bytes:
    return hash_secret_raw(
        secret=password.encode(),
        salt=salt,
        time_cost=3,
        memory_cost=65536,
        parallelism=2,
        hash_len=KEY_LEN,
        type=Type.ID,
    )

## Encryption Pipeline

**Steps:**
1. Read plaintext file
2. Generate random salt
3. Derive key using Argon2
4. Generate AES-GCM nonce
5. Encrypt and authenticate data
6. Store `salt || nonce || ciphertext`


In [ ]:
def encrypt(input_path, output_path, password):
    with open(input_path, "rb") as f:
        data = f.read()

    salt = os.urandom(SALT_LEN)
    key = derive_key(password, salt)

    nonce = os.urandom(NONCE_LEN)
    aesgcm = AESGCM(key)
    ciphertext = aesgcm.encrypt(nonce, data, None)

    with open(output_path, "wb") as f:
        f.write(salt + nonce + ciphertext)

## Decryption Pipeline

Decryption reverses the encryption process:
- Extract salt and nonce
- Re-derive key using the password
- Decrypt and authenticate ciphertext

**Integrity Guarantee:**
AES-GCM ensures that any modification to the ciphertext results in authentication failure.

### 📌 Outcome

This notebook demonstrates:
- Cryptographic correctness
- Security-aware design decisions
- Clean, auditable encryption logic

It validates VaultX as a serious cryptographic engineering project.